In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

## International Matches from tournament software

In [38]:
Tournament_match_links = ['https://bwf.tournamentsoftware.com/sport/matches.aspx?id=2F225B70-FF6C-4BBD-A09E-24EB3A5ACB48',
                         'https://bwf.tournamentsoftware.com/sport/matches.aspx?id=2F225B70-FF6C-4BBD-A09E-24EB3A5ACB48&d=20180208',
                         'https://bwf.tournamentsoftware.com/sport/matches.aspx?id=2F225B70-FF6C-4BBD-A09E-24EB3A5ACB48&d=20180209',
                         'https://bwf.tournamentsoftware.com/sport/matches.aspx?id=2F225B70-FF6C-4BBD-A09E-24EB3A5ACB48&d=20180210']
Tournament_Name = 'Algeria International 2018'
Tournament_Date = '20180207'


In [39]:
#Lists to append
player_list = []
score_list = []
event_list = []
country_list = []
location = ''
for url in Tournament_match_links:
    #Get page session
    r_obj = requests.Session()
    fr_soup = r_obj.get(url)
    soup = BeautifulSoup(fr_soup.content , "lxml")
    location = soup.find_all('span', {'class' : 'hgroup__subheading'})[0].contents[2].split('|')[1].strip(' ')
    detail_tab = soup.find_all("tbody")
    detail_tab = detail_tab[0].find_all("tr")
    
    #Loop through the rows of table
    for row in tqdm(detail_tab):
        if len(row) == 13:
            player_rows = row.find_all('a', href=re.compile('^player'))
            players = []
            for player in player_rows:
                players.append(player.contents[0])
            player_list.append(players)
            
            #Check if score is there
            if row.find_all('span', {'class' : 'score'}):
                score_rows = row.find_all('span', {'class' : 'score'})[0].find_all('span')
                scores = []
                for score in score_rows:
                    scores.append(score.contents[0])
                score_list.append(scores)
            else:
                score_list.append([])
            
            country_rows = row.find_all('span', {'class' : 'printonly flag'})
            countries = []
            for country in country_rows:
                countries.append(country.contents[0])
            country_list.append(countries)

            event_list.append(row.find_all('a', href=re.compile('draw'))[0].contents[0])    
    
    #Close session
    r_obj.close()
    
df = pd.DataFrame()
df['PLAYERS'] = player_list
df['COUNTRIES'] = country_list
df['SCORES'] = score_list
df['EVENT'] = event_list
df['match_week'] = [Tournament_Date]*len(df)
df['match_location'] = [location]*len(df)

df.to_csv('2018 Tournaments/' + str(Tournament_Name) + '_' + str(Tournament_Date) + '.csv')

100%|██████████| 22/22 [00:00<00:00, 1864.70it/s]


IndexError: list index out of range

In [33]:
df

,PLAYERS,COUNTRIES,SCORES,EVENT,match_week,match_location
0,"[Mariano Velarde Alzamora, Jhoey Tejada Rojas]","[[PER] , [PER] ]","[21-16, 21-19]",MS - Qualification,20180517,"Lima, Peru"
1,"[Daniela Zapata, Linda Bu]","[[PER] , [BOL] ]","[21-3, 21-3]",WS,20180517,"Lima, Peru"
2,"[Micaela Castillo Salazar, Andrea Flores Vasqu...","[[PER] , [PER] ]","[21-17, 11-21, 21-17]",WS,20180517,"Lima, Peru"
3,"[Stefany Ying Chie Chen Chen, Iona Gualdi]","[[PER] , [ARG] ]","[21-7, 23-21]",WS,20180517,"Lima, Peru"
4,"[Diana Corleto Soto, Valeria Carolina Wong Lam]","[[GUA] , [PER] ]","[21-15, 21-12]",WS,20180517,"Lima, Peru"
5,"[Rafaela Munar Solimano, Barbara Maria Berruezo]","[[PER] , [ARG] ]","[21-17, 21-18]",WS,20180517,"Lima, Peru"
6,"[Ines Mendoza Rosell, Mariana Isabel Paiz Quan]","[[PER] , [GUA] ]","[22-20, 21-17]",WS,20180517,"Lima, Peru"
7,"[Sara Michele Barrios Chiong, Ariana Moromisat...","[[GUA] , [PER] ]","[21-16, 21-12]",WS,20180517,"Lima, Peru"
8,"[Fabiana Silva, Fernanda Munar Solimano]","[[BRA] , [PER] ]","[21-5, 21-7]",WS,20180517,"Lima, Peru"
9,"[Isabelle Rusli, Micaela Flores Vasquez De Vel...","[[USA] , [PER] ]","[16-21, 21-15, 21-13]",WS,20180517,"Lima, Peru"


In [275]:
r_obj.close()

In [271]:
if row.find_all('span', {'class' : 'score'}):
    print ('hi')

hi


In [240]:
df = pd.DataFrame()
df['PLAYERS'] = player_list
df['COUNTRIES'] = country_list
df['SCORES'] = score_list
df['EVENT'] = event_list
df['match_week'] = [Tournament_Date]*len(df)

df.to_csv('2018 Tournaments/' + str(Tournament_Name) + '_' + str(Tournament_Date) + '.csv')

In [222]:
Tournament_Name

'Estonian_International_2018'

## For just one page

In [2]:
r_obj = requests.Session()
url = "https://bwf.tournamentsoftware.com/sport/matches.aspx?id=A03A06FB-E57B-487D-8866-ADC83F07997F&d=20170614"
fr_soup = r_obj.get(url)
soup = BeautifulSoup(fr_soup.content , "lxml")

In [4]:
detail_tab = soup.find_all("tbody")

In [11]:
a = soup.find_all('span', {'class' : 'hgroup__subheading'})


In [29]:
location = soup.find_all('span', {'class' : 'hgroup__subheading'})[0].contents[2].split('|')[1].strip(' ')

In [30]:
location

'Jakarta, Indonesia'

In [3]:
detail_tab = soup.find_all("tbody")
detail_tab = detail_tab[0].find_all("tr")

In [205]:
#Lists to append
player_list = []
score_list = []
event_list = []
country_list = []

for row in tqdm(detail_tab):
    if len(row) == 13:
        player_rows = row.find_all('a', href=re.compile('^player'))
        players = []
        for player in player_rows:
            players.append(player.contents[0])
        player_list.append(players)
        
        score_rows = row.find_all('span', {'class' : 'score'})[0].find_all('span')
        scores = []
        for score in score_rows:
            scores.append(score.contents[0])
        score_list.append(scores)
        
        country_rows = row.find_all('span', {'class' : 'printonly flag'})
        countries = []
        for country in country_rows:
            countries.append(country.contents[0])
        country_list.append(countries)
        
        event_list.append(row.find_all('a', href=re.compile('draw'))[0].contents[0])

100%|██████████| 203/203 [00:00<00:00, 2107.19it/s]


In [206]:
df = pd.DataFrame()
df['PLAYERS'] = player_list
df['COUNTRIES'] = country_list
df['SCORES'] = score_list
df['EVENT'] = event_list

In [207]:
df

,PLAYERS,COUNTRIES,SCORES,EVENT
0,"[Huang Yaqiong, Yu Xiaohan, Misaki Matsutomo [...","[[CHN] , [CHN] , [JPN] , [JPN] ]","[21-7, 22-20]",WD
1,"[Kim Astrup, Anders Skaarup Rasmussen, Marcus ...","[[DEN] , [DEN] , [INA] , [INA] ]","[21-16, 21-16]",MD
2,"[Yuki Fukushima, Sayaka Hirota, Serena Kani, J...","[[JPN] , [JPN] , [INA] , [INA] ]","[21-10, 21-14]",WD
3,"[Liu Cheng, Zhang Nan, Ong Yew Sin, Teo Ee Yi]","[[CHN] , [CHN] , [MAS] , [MAS] ]","[21-17, 21-12]",MD
4,"[Shiho Tanaka [8], Koharu Yonemoto, Yin Loo Li...","[[JPN] , [JPN] , [MAS] , [MAS] ]","[21-18, 14-21, 21-12]",WD
5,"[Naoko Fukuman, Kurumi Yonao, Keshya Nurvita H...","[[JPN] , [JPN] , [INA] , [INA] ]","[21-10, 21-14]",WD
6,"[Chang Ye Na [3], Lee So Hee, Maiken Fruergaar...","[[KOR] , [KOR] , [DEN] , [DEN] ]","[21-14, 21-13]",WD
7,"[Takeshi Kamura [6], Keigo Sonoda, Chen Hung L...","[[JPN] , [JPN] , [TPE] , [TPE] ]","[21-18, 14-10]",MD
8,"[Greysia Polii, Apriyani Rahayu, Chae Yoo Jung...","[[INA] , [INA] , [KOR] , [KOR] ]","[15-21, 25-23, 21-14]",WD
9,"[Mathias Christiansen, David Daugaard, Takuro ...","[[DEN] , [DEN] , [JPN] , [JPN] ]","[21-16, 21-13]",MD
